In [1]:
import csv
import numpy as np
import pandas as pd
import collections

In [3]:
keys = pd.read_csv('nfl-tweets-v1.0/tweets.key.csv')
tweet_id_df = pd.read_csv('nfl-tweets-v1.0/tweets.nfl.2012.weekly.csv', names=list(keys))

In [4]:
tweet_id_df.head()

,tweet_id,tweet_UTCtime,team,opponent,week,home_away,score,opponent_score,point_spread,over_under
0,240600003763113984,2012-08-29T00:01:25,DAL,NYG,1,AWAY,24,17,3.0,47.0
1,240600016358604801,2012-08-29T00:01:28,TB,CAR,1,HOME,16,10,2.5,47.0
2,240600029197398016,2012-08-29T00:01:31,PHI,CLE,1,AWAY,17,16,-9.0,42.0
3,240600071073308672,2012-08-29T00:01:42,CHI,IND,1,HOME,41,21,-9.5,41.5
4,240600083685588993,2012-08-29T00:01:44,NYG,DAL,1,HOME,17,24,-3.0,47.0


In [59]:
len(tweet_id_df.loc[tweet_id_df['week'] == 11]['team'].unique())

28

In [24]:
# Keep track of all games played for each week
num_weeks = 16

games = {i: set() for i in range(1, num_weeks+1)}

for wk in range(1, num_weeks + 1):
    wk_games = tweet_id_df.loc[(tweet_id_df['home_away'] == 'HOME') & (tweet_id_df['week'] == wk)]
    home_teams = list(wk_games['team'])
    away_teams = list(wk_games['opponent'])
    home_score = list(wk_games['score'])
    away_score = list(wk_games['opponent_score'])

    for i in range(len(home_teams)):
        home_team = home_teams[i]
        away_team = away_teams[i]
        win = 0

        # Not handling ties here, just giving home team the win if theres a tie
        if home_score[i] >= away_score[i]:
            win = 1
        games[wk].add((home_team, away_team, win))


In [56]:
print(len(games[11]))

14


In [7]:
pass_df = pd.read_csv('2012-passing-yards-per-game.csv')
points_df = pd.read_csv('2012-points-per-game.csv')
rush_df = pd.read_csv('2012-rushing-yards-per-game.csv')
turnover_df = pd.read_csv('2012-turnover-margin-per-game.csv')

In [13]:
pass_df.shape

(512, 10)

In [14]:
points_df.head()

,Rank,Team,2012,Last 3,Last 1,Home,Away,2011,Week,Team_Abb
0,NaN,Atlanta,--,--,--,--,--,23.8,1,ATL
1,NaN,New Orleans,--,--,--,--,--,34.7,1,NO
2,NaN,NY Giants,--,--,--,--,--,24.8,1,NYG
3,NaN,NY Jets,--,--,--,--,--,23.6,1,NYJ
4,NaN,Philadelphia,--,--,--,--,--,24.8,1,PHI


In [18]:
turnover_df.tail(20)

,Rank,Team,2012,Last 3,Last 1,Home,Away,2011,Week,Team_Abb
492,13.0,Cincinnati,+0.2,+0.7,+3.0,+0.9,-0.4,-0.2,16,CIN
493,13.0,Carolina,+0.2,+0.7,+1.0,0.0,+0.4,0.1,16,CAR
494,13.0,Arizona,+0.2,-0.3,+3.0,+0.6,-0.1,-0.8,16,ARI
495,16.0,Jacksonville,+0.1,+0.3,+1.0,-0.7,+0.9,0.3,16,JAX
496,16.0,New Orleans,+0.1,-0.3,+5.0,+0.7,-0.6,-0.4,16,NO
497,18.0,Denver,0.0,+1.0,+2.0,+0.2,-0.1,-0.6,16,DEN
498,19.0,LA Chargers,-0.1,+0.7,-1.0,-0.6,+0.4,-0.4,16,SD
499,20.0,LA Rams,-0.2,0.0,-2.0,-0.7,+0.3,-0.3,16,STL
500,20.0,Minnesota,-0.2,+0.7,+2.0,+0.1,-0.6,-0.2,16,MIN
501,22.0,Tennessee,-0.4,-0.3,+5.0,-1.0,+0.3,0.1,16,TEN


In [16]:
rush_df.tail()

,Rank,Team,2012,Last 3,Last 1,Home,Away,2011,Week,Team_Abb
507,28.0,Atlanta,89.9,96.0,129.0,95.3,84.6,111.6,16,ATL
508,29.0,Las Vegas,89.9,116.3,203.0,90.5,89.0,131.9,16,OAK
509,30.0,Jacksonville,82.4,86.3,86.0,74.0,90.9,123.1,16,JAX
510,31.0,Dallas,80.4,86.3,87.0,58.0,102.7,112.9,16,DAL
511,32.0,Arizona,80.2,74.3,99.0,76.0,84.4,101.6,16,ARI


In [68]:
all_game_data_headers = ['home', 'away', 'week', 'home_pass', 'away_pass', 'home_rush', 'away_rush', 'home_points', 'away_points', 'home_turnover', 'away_turnover', 'home_win']
all_game_data = []
for wk, game_set in games.items():
    for game in list(game_set):
        home = game[0]
        away = game[1]
        win = game[2]

        # Grab home pass data and if the value is null (first game), then use the previous year %
        home_pass = pass_df.loc[(pass_df['Team_Abb'] == home) & (pass_df['Week'] == wk)]['Last 3'].values[0]
        if home_pass == '--':
            home_pass = pass_df.loc[(pass_df['Team_Abb'] == home) & (pass_df['Week'] == wk)]['2011'].values[0]

        away_pass = pass_df.loc[(pass_df['Team_Abb'] == away) & (pass_df['Week'] == wk)]['Last 3'].values[0]
        if away_pass == '--':
            away_pass = pass_df.loc[(pass_df['Team_Abb'] == away) & (pass_df['Week'] == wk)]['2011'].values[0]

        # Grab home rush data and if the value is null (first game), then use the previous year %
        home_rush = rush_df.loc[(rush_df['Team_Abb'] == home) & (rush_df['Week'] == wk)]['Last 3'].values[0]
        if home_rush == '--':
            home_rush = rush_df.loc[(rush_df['Team_Abb'] == home) & (rush_df['Week'] == wk)]['2011'].values[0]

        away_rush = rush_df.loc[(rush_df['Team_Abb'] == away) & (rush_df['Week'] == wk)]['Last 3'].values[0]
        if away_rush == '--':
            away_rush = rush_df.loc[(rush_df['Team_Abb'] == away) & (rush_df['Week'] == wk)]['2011'].values[0]

        # Grab home points data and if the value is null (first game), then use the previous year %
        home_points = points_df.loc[(points_df['Team_Abb'] == home) & (points_df['Week'] == wk)]['Last 3'].values[0]
        if home_points == '--':
            home_points = points_df.loc[(points_df['Team_Abb'] == home) & (points_df['Week'] == wk)]['2011'].values[0]

        away_points = points_df.loc[(points_df['Team_Abb'] == away) & (points_df['Week'] == wk)]['Last 3'].values[0]
        if away_points == '--':
            away_points = points_df.loc[(points_df['Team_Abb'] == away) & (points_df['Week'] == wk)]['2011'].values[0]

        # Grab home points data and if the value is null (first game), then use the previous year %
        home_turnover = turnover_df.loc[(turnover_df['Team_Abb'] == home) & (turnover_df['Week'] == wk)]['Last 3'].values[0]
        if home_turnover == '--':
            home_turnover = turnover_df.loc[(turnover_df['Team_Abb'] == home) & (turnover_df['Week'] == wk)]['2011'].values[0]

        away_turnover = turnover_df.loc[(turnover_df['Team_Abb'] == away) & (turnover_df['Week'] == wk)]['Last 3'].values[0]
        if away_turnover == '--':
            away_turnover = turnover_df.loc[(turnover_df['Team_Abb'] == away) & (turnover_df['Week'] == wk)]['2011'].values[0]

        game_data = [home_pass, away_pass, home_rush, away_rush, home_points, away_points, home_turnover, away_turnover]
        game_data = list(map(float, game_data))
        game_data.insert(0, wk)
        game_data.insert(0, away)
        game_data.insert(0, home)
        game_data.append(win)

        all_game_data.append(game_data)

        # print(game_data)
print(len(all_game_data))

240


In [69]:
all_game_df = pd.DataFrame(all_game_data, columns=all_game_data_headers)

In [72]:
all_game_df.head()

,home,away,week,home_pass,away_pass,home_rush,away_rush,home_points,away_points,home_turnover,away_turnover,home_win
0,MIN,JAX,1,184.8,136.2,144.9,123.1,21.3,15.2,-0.2,0.3,1
1,KC,ATL,1,192.5,257.4,118.3,111.6,13.3,23.8,-0.1,0.5,0
2,NYG,DAL,1,293.9,262.6,94.7,112.9,24.8,23.1,0.7,0.3,0
3,NO,WAS,1,346.7,235.8,129.5,100.9,34.7,18.0,-0.4,-0.9,0
4,OAK,SD,1,247.6,276.6,131.9,116.5,22.4,25.4,-0.3,-0.4,0


In [73]:
all_game_df.to_csv('all_game_data.csv', index=False)